# ICI - generate features and labels
1. Iterate through ICI cohort
2. For each MRN, find the most proximal ECG in year prior to ICI (or for controls, "event date")
3. Get features
4. Label is "ICI exposure"

In [ ]:
import os
import h5py
import numcodecs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from tqdm import tqdm
import dateutil.parser
from datetime import datetime, timedelta

In [ ]:
def decompress_data(data_compressed: np.array, dtype: str) -> np.array:
    """Decompresses a compressed byte array. If the primitive type of the data
    to decompress is a string, calls decode using the zstd codec. If the
    primitive type of the data to decompress is not a string (e.g. int or
    float), the buffer is interpreted using the passed dtype."""
    codec = numcodecs.zstd.Zstd()
    data_decompressed = codec.decode(data_compressed)
    if dtype == 'str':
        data = data_decompressed.decode()
    else:
        data = np.frombuffer(data_decompressed, dtype)
    return data

def get_feature_from_hd5(hd5, key: str):
    data = np.nan
    if key in hd5:
        data = decompress_data(data_compressed=hd5[key][()], dtype=hd5[key].attrs['dtype'])
    return data

features_from_ecg = [
'patientage',
'paxis_md',
'qoffset_md',
'qonset_md',
'qrscount_md',
'qrsduration_md',
'qtcorrected_md',
'qtinterval_md',
'raxis_md',
'taxis_md',
'toffset_md',
'ventricularrate_md']

# 'measuredamplitudearea_IE_Q',
# 'measuredamplitudearea_IE_R',
# 'measuredamplitudearea_IE_RPM',
# 'measuredamplitudearea_IE_S',
# 'measuredamplitudearea_IE_SPM',
# 'measuredamplitudearea_IE_STJ',
# 'measuredamplitudearea_IE_STJ40',
# 'measuredamplitudearea_IE_STJ60',
# 'measuredamplitudearea_IE_STJ80',
# 'measuredamplitudearea_IE_T',
# 'measuredamplitudearea_IE_TPM',
# 'measuredamplitudeduration_IE_Q',
# 'measuredamplitudeduration_IE_R',
# 'measuredamplitudeduration_IE_RPM',
# 'measuredamplitudeduration_IE_S',
# 'measuredamplitudeduration_IE_SPM',
# 'measuredamplitudeduration_IE_STJ',
# 'measuredamplitudeduration_IE_STJ40',
# 'measuredamplitudeduration_IE_STJ60',
# 'measuredamplitudeduration_IE_STJ80',
# 'measuredamplitudeduration_IE_T',
# 'measuredamplitudeduration_IE_TPM',
# 'measuredamplitudepeak_IE_Q',
# 'measuredamplitudepeak_IE_R',
# 'measuredamplitudepeak_IE_RPM',
# 'measuredamplitudepeak_IE_S',
# 'measuredamplitudepeak_IE_SPM',
# 'measuredamplitudepeak_IE_STJ',
# 'measuredamplitudepeak_IE_STJ40',
# 'measuredamplitudepeak_IE_STJ60',
# 'measuredamplitudepeak_IE_STJ80',
# 'measuredamplitudepeak_IE_T',
# 'measuredamplitudepeak_IE_TPM',
# 'measuredamplitudestart_IE_Q',
# 'measuredamplitudestart_IE_R',
# 'measuredamplitudestart_IE_RPM',
# 'measuredamplitudestart_IE_S',
# 'measuredamplitudestart_IE_SPM',
# 'measuredamplitudestart_IE_STJ',
# 'measuredamplitudestart_IE_STJ40',
# 'measuredamplitudestart_IE_STJ60',
# 'measuredamplitudestart_IE_STJ80',
# 'measuredamplitudestart_IE_T',
# 'measuredamplitudestart_IE_TPM',

In [ ]:
# Get ICI cohort from CSV to DataFrame
path_to_ici_cohort = os.path.expanduser("~/dropbox/ecg_immunotherapy/data/casecontrol2020624_events.csv")
df_ici = pd.read_csv(path_to_ici_cohort)

In [ ]:
# Isolate only cases; remove controls
df_ici = df_ici[df_ici.group == "case"]
df_ici

In [ ]:
# Get list of full paths to HD5 files
path_to_tensors = "/data/ecg/mgh"
mrns = []
tensors = []
for root, dirs, files in os.walk(path_to_tensors):
    for file in files:
        if file.endswith(".hd5"):
            tensors.append(os.path.join(root, file))
            mrns.append(int(file.replace(".hd5", "")))
print(f"Obtained paths to {len(tensors)} hd5 files")

In [ ]:
# Get ECGs at intersect
mrns_intersect = set(mrns).intersection(set(df_ici.mgh_mrn.values))

ecg_features_before = []
ecg_features_after = []

# Iterate through MRNs at intersect
for mrn in tqdm(mrns_intersect):
    fpath = os.path.join(path_to_tensors, str(mrn) + ".hd5")
    keys = None
    
    # Get date of event
    event_date_str = df_ici[df_ici.mgh_mrn == mrn].ici_start_for_calc.values[0]
    event_date = datetime.strptime(event_date_str, "%m/%d/%y")
    
    # Define time window from [event_date - 1 yr, event_date]

    with h5py.File(fpath, 'r') as hf:
        keys = list(hf['partners_ecg_rest'].keys())
        dates_list = [dateutil.parser.parse(date) for date in keys]
        
        # Get diffs between each ECG and the event date
        diffs_in_days = np.array([(date - event_date).days for date in dates_list])
        
        # Index of min preop ECG
        if np.any(diffs_in_days < 0):
            
            idx_diff = np.argmin(np.abs(diffs_in_days[diffs_in_days < 0]))

            # Diff at this index
            diff_of_closest_preop_ecg = diffs_in_days[diffs_in_days < 0][idx_diff]

            if diff_of_closest_preop_ecg < 365:

                # Initialize new features dict
                features_this_ecg = {}
                
                # Index of ECG with this diff
                idx_ecg = np.argwhere(diffs_in_days == diff_of_closest_preop_ecg)[0][0]
                
                # Get pointer to ECG
                ecg = hf['partners_ecg_rest'][keys[idx_ecg]]

                # Iterate through list of feature names and get them from ECG
                for feature in features_from_ecg:
                    features_this_ecg[feature] = get_feature_from_hd5(
                        hd5=ecg, key=feature)
                
                features_this_ecg['mrn'] = mrn
                
                features_this_ecg['ecg_date'] = keys[idx_ecg]
                
                # Append dict of features to list
                ecg_features_before.append(features_this_ecg)
                
        # Index of min preop ECG
        if np.any(diffs_in_days > 0):
            
            idx_diff = np.argmin(np.abs(diffs_in_days[diffs_in_days > 0]))

            # Diff at this index
            diff_of_closest_preop_ecg = diffs_in_days[diffs_in_days > 0][idx_diff]

            if diff_of_closest_preop_ecg < 365:

                # Initialize new features dict
                features_this_ecg = {}
                
                # Index of ECG with this diff
                idx_ecg = np.argwhere(diffs_in_days == diff_of_closest_preop_ecg)[0][0]

                # Get pointer to ECG
                ecg = hf['partners_ecg_rest'][keys[idx_ecg]]

                # Iterate through list of feature names and get them from ECG
                for feature in features_from_ecg:
                    features_this_ecg[feature] = get_feature_from_hd5(
                        hd5=ecg, key=feature)
                
                features_this_ecg['mrn'] = mrn
                
                features_this_ecg['ecg_date'] = keys[idx_ecg]
                
                # Append dict of features to list
                ecg_features_after.append(features_this_ecg)
                
print(f"Found {len(ecg_features_before)} MRNs with an ECG in year before event")
print(f"Found {len(ecg_features_after)} MRNs with an ECG in year aftert event")

In [ ]:
# Convert list of dicts into dataframe
df_ecg_features_before = pd.DataFrame(ecg_features_before)
df_ecg_features_before

In [ ]:
# Convert list of dicts into dataframe
df_ecg_features_after = pd.DataFrame(ecg_features_after)
df_ecg_features_after

In [ ]:
# Rename mgh_mrn to mrn column
df_ici = df_ici.rename(columns={'mgh_mrn':'mrn'})
df_ici

In [ ]:
# Join ICI dataframe with ECG features
df_merge_before = df_ici.merge(df_ecg_features_before, on='mrn')
df_merge_after = df_ici.merge(df_ecg_features_after, on='mrn')

In [ ]:
sns.set(style="white", context="talk", palette="muted", color_codes=True)
for feature in features_from_ecg:
    plt.figure(figsize=(4, 3), dpi=100)
    g1 = sns.distplot(df_merge_before[feature], color="r")
    g2 = sns.distplot(df_merge_after[feature], color="b")
    sns.despine(left=True)
    ax = plt.gcf().axes
    g1.set(yticks=[])
    plt.tight_layout()